In [1]:
from fastai.vision import * #import from vision to use the classification interpreter
from pathlib import Path
import pdb
import seaborn as sns
from sklearn.decomposition import PCA
from timeseries import TimeSeriesItem, TimeSeriesList, UCRArchive

In [2]:
ucr = UCRArchive()

In [3]:
def create_head_1d(nf:int, nc:int, lin_ftrs:Optional[Collection[int]]=None, ps:Floats=0.5, bn_final:bool=False):
    "Model head that takes `nf` features, runs through `lin_ftrs`, and about `nc` classes."
    lin_ftrs = [nf, 512, nc] if lin_ftrs is None else [nf] + lin_ftrs + [nc]
    ps = listify(ps)
    if len(ps)==1: ps = [ps[0]/2] * (len(lin_ftrs)-2) + ps
    actns = [nn.ReLU(inplace=True)] * (len(lin_ftrs)-2) + [None]
    layers = []
    for ni,no,p,actn in zip(lin_ftrs[:-1],lin_ftrs[1:],ps,actns):
        layers += bn_drop_lin(ni,no,True,p,actn)
    if bn_final: layers.append(nn.BatchNorm1d(lin_ftrs[-1], momentum=0.01))
    return nn.Sequential(*layers)

def conv1d(ni:int, nf:int, ks:int=3, stride:int=1, padding:int=None, bias=False, init:LayerFunc=nn.init.kaiming_normal_) -> nn.Conv1d:
    "Create and initialize `nn.Conv1d` layer. `padding` defaults to `ks//2`."
    if padding is None: padding = ks//2
    return init_default(nn.Conv1d(ni, nf, kernel_size=ks, stride=stride, padding=padding, bias=bias), init)

def _bn1d(ni, init_zero=False):
    "Batchnorm layer with 0 initialization"
    m = nn.BatchNorm1d(ni)
    m.weight.data.fill_(0 if init_zero else 1)
    m.bias.data.zero_()
    return m

def bn_relu_conv1d(ni, nf, ks, stride, init_zero=False):
    bn_initzero = _bn1d(ni, init_zero=init_zero)
    return nn.Sequential(bn_initzero, nn.ReLU(inplace=True), conv1d(ni, nf, ks, stride))

class ResBlock(torch.nn.Module):
    def __init__(self, ni, nf, stride, drop_p=0.0):
        super().__init__()
        self.bn = nn.BatchNorm1d(ni)
        self.conv1 = conv1d(ni, nf, 3, stride)
        self.conv2 = bn_relu_conv1d(nf, nf, 3, 1)
        self.drop = nn.Dropout(drop_p, inplace=True) if drop_p else None
        self.shortcut = conv1d(ni, nf, 1, stride) if ni != nf or stride > 1 else noop

    def forward(self, x):
        x2 = F.relu(self.bn(x), inplace=True)
        r = self.shortcut(x2)
        x = self.conv1(x2)
        if self.drop: x = self.drop(x)
        x = self.conv2(x) * 0.2
        return x.add_(r)

def _make_group(N, ni, nf, block, stride, drop_p):
    return [block(ni if i == 0 else nf, nf, stride if i == 0 else 1, drop_p) for i in range(N)]

class WideResNet1d(nn.Module):
    "Wide ResNet with `num_groups` and a width of `k`."
    def __init__(self, num_groups:int, N:int, k:int=1, drop_p:float=0.0, start_nf:int=16,maxY=20.0):
        super().__init__()
        self.maxY = maxY
        n_channels = [start_nf]
        for i in range(num_groups): n_channels.append(start_nf*(2**i)*k)
        
        
        layers = [conv1d(1, n_channels[0], 3, 1)]  # conv1
        self.split_groups = [layers[-1]] 
        
        for i in range(num_groups):
            layers += _make_group(N, n_channels[i], n_channels[i+1], ResBlock, (1 if i==0 else 2), drop_p)
            self.split_groups.append(layers[-N])
            
        layers += [nn.BatchNorm1d(n_channels[-1]), nn.ReLU(inplace=True), nn.AdaptiveAvgPool1d(1),
                   Flatten()]
        #self.split_groups.append(layers[-1])
        self.nf = n_channels[-1]
        self.features = nn.Sequential(*layers)

    def forward(self, x):
        #x = x.unsqueeze(1)
        return self.features(x)

class HeroResnet(nn.Module):
    def __init__(self, num_groups:int, N:int, k:int=1, drop_p:float=0.0, start_nf:int=16):
        super().__init__()
        n_channels = [start_nf]
        for i in range(num_groups): n_channels.append(start_nf*(2**i)*k)

        self.layer0 = conv1d(1,n_channels[0],3,1)
        self.split_groups = [self.layer0]
        
        groups = [nn.Sequential(*_make_group(N,n_channels[i],n_channels[i+1],ResBlock,(1 if i==0 else 2),drop_p)) for i in range(num_groups)]
        
        self.groups = nn.ModuleList(groups)
        for g in self.groups: self.split_groups.append(g)
        
        self.out = nn.Sequential(nn.BatchNorm1d(n_channels[-1]),
                                 nn.ReLU(inplace=True),
                                 nn.AdaptiveAvgPool1d(1))               
        
        self.split_groups.append(self.out)
        self.nf = n_channels[-1]
        
    def forward(self, x):
        actvns = [self.layer0(x)]
        
        for l in self.groups:
            actvns.append(l(actvns[-1]))
            
        return self.out(actvns[-1]), actvns

class SidekickResnet(nn.Module):
    def __init__(self, num_classes:int, num_groups:int, N:int, k:int=1, hero_k:int=1, drop_p:float=0.0, start_nf:int=16, start_nf_hero:int=16):
        super().__init__()
        n_channels = [start_nf]
        
        self.hero = HeroResnet(num_groups,N,hero_k,drop_p,start_nf_hero)
        
        for i in range(num_groups): n_channels.append(start_nf*(2**i)*k)
            
        n_channels_hero = [start_nf_hero]
        for i in range(num_groups): n_channels_hero.append(start_nf_hero*(2**i)*hero_k)   
            
        self.layer0 = conv1d(1,n_channels[0],3,1)
        
        groups = [nn.Sequential(*_make_group(N,n_channels[i]+n_channels_hero[i],n_channels[i+1],ResBlock,(1 if i==0 else 2),drop_p)) for i in range(num_groups)]

        
        self.groups = nn.ModuleList(groups)
        
        self.avg = nn.Sequential(nn.BatchNorm1d(n_channels[-1]),
                                 nn.ReLU(inplace=True),
                                 nn.AdaptiveAvgPool1d(1))
        
        self.out = create_head_1d(n_channels[-1]+n_channels_hero[-1],num_classes,ps=0.5)
        
    def forward(self,ts):
        ts = ts.unsqueeze(1)
        pt, actvns = self.hero(ts)
        
        x = self.layer0(ts)
        for l,a in zip(self.groups,actvns):
            x = l(torch.cat([x,a],dim=1))
            
        x = torch.cat([self.avg(x),pt],dim=1).squeeze(-1)
        
        return self.out(x)    

In [4]:
class TSAE(torch.nn.Module):
    def __init__(self,seqLen,latentDim=24):
        super().__init__()
        self.conv = HeroResnet(3,3,12,drop_p=0.3)
        self.mean = torch.nn.Linear(self.conv.nf,latentDim)
        self.logvar = torch.nn.Linear(self.conv.nf,latentDim)
        

        layers = []
        for a,b in [(latentDim,100),(100,200),(200,300)]:
            layers += bn_drop_lin(a,b,actn=torch.nn.ReLU())
        self.lin = torch.nn.Sequential(*layers)
        self.out = torch.nn.Linear(300,seqLen)

    def forward(self,ts):
        seqLen = ts.shape[1]
        ts, _ = self.conv(ts.unsqueeze(1))
        ts = ts.squeeze(-1)
        mean, logvar = self.mean(ts), self.logvar(ts)
        #mean = self.mean(ts)
        
        ls = mean
        if self.training:
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            ls = eps.mul(std).add_(mean)
        #return self.out(self.lin(ls))          
        return self.out(self.lin(ls)), mean, logvar

class VAELoss(torch.nn.Module):
    def forward(self,p,target):
        pred,mean,logvar = p
        self.mse = torch.nn.functional.mse_loss(pred,target,reduction="sum")
        self.kld = -0.5 * torch.sum(1+logvar-mean.pow(2)-logvar.exp())
        return self.mse + self.kld

In [5]:
def EvaluateSeries(dataset_name):
    out = [dataset_name]
    print(f"Evaluating {dataset_name}")
    try:
        src = TimeSeriesList.from_csv_list(ucr.get_csv_files(dataset_name),labelCol=0)
        valIdxs = np.random.choice(len(src.items),int(len(src.items)*0.3),replace=False)
        data = src.split_by_idx(valIdxs)
        data = data.label_from_col()
        idxs = np.random.choice(len(data.x),size=len(data.x)//10,replace=False)
        bs = min(64,len(data.x)//50)
        data.x.items = data.train.x.items[idxs]
        data.y.items = data.train.y.items[idxs]
        data = data.databunch(bs=bs,num_workers=0)

        src = TimeSeriesList.from_csv_list(ucr.get_csv_files(dataset_name),labelCol=0)
        dataAE = src.split_by_idx(valIdxs)
        dataAE = dataAE.label_from_self()
        dataAE = dataAE.databunch(bs=bs,num_workers=0)
        
        seqLen = len(data.train_ds[0][0].data)
        
        learnAE = Learner(dataAE,TSAE(len(data.train_ds[0][0].data),max(10,seqLen//8)),loss_func=VAELoss())
        learnAE.fit_one_cycle(10,1e-2)
        learnAE.fit_one_cycle(10,1e-3)
        out.append(learnAE.validate(dataAE.train_dl)[0])
        
        learn = Learner(data,SidekickResnet(data.train_ds.c,3,3,6,12), loss_func=F.cross_entropy,metrics=[accuracy],callback_fns=BnFreeze,bn_wd=False,train_bn=False)
        learn.split(split_model(learn.model,[learn.model.hero,learn.model.layer0,learn.model.avg,learn.model.out]))
        learn.fit_one_cycle(20,1e-2,wd=0.2)
        learn.fit_one_cycle(20,1e-3,wd=0.2)
        out.append(max([m[0].item() for m in learn.recorder.metrics]))
        
        learn = Learner(data,SidekickResnet(data.train_ds.c,3,3,6,12), loss_func=F.cross_entropy,metrics=[accuracy],callback_fns=BnFreeze,bn_wd=False,train_bn=False)
        learn.split(split_model(learn.model,[learn.model.hero,learn.model.layer0,learn.model.avg,learn.model.out]))
        learn.model.hero.load_state_dict(learnAE.model.conv.state_dict())
        learn.freeze_to(1)
        learn.fit_one_cycle(20,1e-2,wd=0.2)
        learn.fit_one_cycle(20,1e-3,wd=0.2)
        out.append(max([m[0].item() for m in learn.recorder.metrics]))

    except:
        pass
    
    return out

In [6]:
results = [EvaluateSeries(d) for d in progress_bar(ucr.list_datasets())]

epoch,train_loss,valid_loss,accuracy
1,1.246314,1.777522,0.359116
2,0.978859,1.745663,0.359116
3,0.902215,2.507949,0.359116
4,0.779319,3.583701,0.359116
5,0.718476,3.255053,0.226519
6,0.964139,265.908142,0.182320
7,0.941253,217.163910,0.314917
8,0.901936,136.438766,0.314917
9,0.884891,33.722542,0.193370
10,0.870115,20.566774,0.314917


epoch,train_loss,valid_loss,accuracy
1,0.405819,0.953251,0.734807
2,0.373216,0.928323,0.745856
3,0.453165,0.933091,0.751381
4,0.389980,0.915465,0.762431
5,0.422400,0.962405,0.751381
6,0.393746,0.876071,0.751381
7,0.348221,0.926337,0.712707
8,0.344904,1.143104,0.574586
9,0.338783,1.407879,0.497238
10,0.378272,1.492332,0.469613


epoch,train_loss,valid_loss,accuracy


Evaluating RefrigerationDevices


epoch,train_loss,valid_loss


Evaluating Rock
Evaluating ScreenType
Evaluating SemgHandGenderCh2
Evaluating SemgHandMovementCh2
Evaluating SemgHandSubjectCh2
Evaluating ShakeGestureWiimoteZ
Evaluating ShapeletSim
Evaluating ShapesAll
Evaluating SmallKitchenAppliances
Evaluating SmoothSubspace
Evaluating SonyAIBORobotSurface1
Evaluating SonyAIBORobotSurface2
Evaluating StarLightCurves
Evaluating Strawberry
Evaluating SwedishLeaf
Evaluating Symbols
Evaluating SyntheticControl
Evaluating ToeSegmentation1
Evaluating ToeSegmentation2
Evaluating Trace
Evaluating TwoLeadECG
Evaluating TwoPatterns
Evaluating UMD
Evaluating UWaveGestureLibraryAll
Evaluating UWaveGestureLibraryX
Evaluating UWaveGestureLibraryY
Evaluating UWaveGestureLibraryZ
Evaluating Wafer
Evaluating Wine
Evaluating WordSynonyms
Evaluating Worms
Evaluating WormsTwoClass
Evaluating Yoga


In [7]:
rDF = pd.DataFrame(results,columns=["Name","AE Loss","Random","Pretrained"])

In [8]:
rDF["GainT"] = rDF["Pretrained"] - rDF["Random"]

In [17]:
rDF.to_csv("ucrresults.csv")

In [3]:
rDF = pd.read_csv("ucrresults.csv")

In [9]:
rDF.sort_values("GainT",ascending=False)

,Name,AE Loss,Random,Pretrained,GainT
89,Plane,1.048508e+02,0.301587,0.650794,0.349206
9,BME,1.245151e+02,0.444444,0.574074,0.129630
0,ACSF1,1.408396e+08,0.233333,0.350000,0.116667
5,ArrowHead,8.808843e+01,0.460317,0.555556,0.095238
1,Adiac,6.057321e+01,0.303419,0.371795,0.068376
90,PowerCons,4.945986e+02,0.796296,0.861111,0.064815
87,PigCVP,1.553118e+05,0.086022,0.096774,0.010753
2,AllGestureWiimoteX,NaN,0.103333,0.103333,0.000000
3,AllGestureWiimoteY,NaN,0.093333,0.093333,0.000000
4,AllGestureWiimoteZ,NaN,0.076667,0.076667,0.000000
